# Student Name: Ganapathy 

# Student ID: 18202799

# Feature Selection

### Preperation

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn import preprocessing 

from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
import io
from google.colab import files

Load and prepare the dataset for training and evaluation

In [5]:
uploaded = files.upload()

Saving winequality_red.csv to winequality_red.csv


In [0]:
df = pd.read_csv(io.BytesIO(uploaded['winequality_red.csv']))

In [7]:
df.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000
mean,8.319637,0.527821,0.270976,2.538806,0.087467,15.874922,46.467792,0.996747,3.311113,0.658149,10.422983,5.636023
std,1.741096,0.179060,0.194801,1.409928,0.047065,10.460157,32.895324,0.001887,0.154386,0.169507,1.065668,0.807569
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,3.000000
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000,5.000000
50%,7.900000,0.520000,0.260000,2.200000,0.079000,14.000000,38.000000,0.996750,3.310000,0.620000,10.200000,6.000000
75%,9.200000,0.640000,0.420000,2.600000,0.090000,21.000000,62.000000,0.997835,3.400000,0.730000,11.100000,6.000000
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000,8.000000


In [0]:
# target the specific column
target_col_name = 'quality'
target = df[target_col_name]

# predictor attributes
predictors = df.drop(target_col_name, axis=1).values

# scale all the predictor values to the range [0, 1]
# note the target attribute is already binary
min_max_scaler = preprocessing.MinMaxScaler()
predictors = min_max_scaler.fit_transform(predictors)

Split the dataset into 80:20 rule

In [0]:
# pepare independent data sets for training and test of the final model
predictors_train, predictors_test, target_train, target_test = train_test_split(predictors, target, test_size=0.20, shuffle=True, stratify=target)

# B. Feature Selection
## 1. Apply RFE with SVM for selecting the best features

In [10]:
# create a base classifier to evaluate a subset of attributes
estSVM = svm.SVR(kernel="linear")
selSVM = RFE(estSVM, 3)
selSVM = selSVM.fit(predictors_train, target_train)
#print the attributes
print(selSVM.support_)
print(selSVM.ranking_)

[False  True False False False False False False False  True  True]
[5 1 8 4 2 7 3 6 9 1 1]


In [11]:
# create a base classifier used to evaluate a subset of attributes
estLR = LogisticRegression()
# create the RFE model and select 3 attributes
selLR = RFE(estLR, 3)
selLR = selLR.fit(predictors_train, target_train)
# summarize the selection of the attributes
print(selLR.support_)
print(selLR.ranking_)

[False  True False False False False  True False False False  True]
[6 1 4 9 8 5 1 3 7 2 1]


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed

## B. Evaluate on the Test Data Set

Apply the selectors to prepare training data sets for the selected featuresNote: The same selectors are applied to the test data set. Test data

In [0]:
predictors_train_SVMselected = selSVM.transform(predictors_train)
predictors_test_SVMselected = selSVM.transform(predictors_test)

In [0]:
 predictors_train_LRselected = selLR.transform(predictors_train)
predictors_test_LRselected = selLR.transform(predictors_test)

In [0]:
classifier = svm.SVC()

In [15]:
 model1 = classifier.fit(predictors_train_SVMselected, target_train)
model1.score(predictors_test_SVMselected, target_test)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0.55

In [16]:
model2 = classifier.fit(predictors_train_LRselected, target_train)
model2.score(predictors_test_LRselected, target_test)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0.55625

In [17]:
model3 = classifier.fit(predictors_train, target_train)
model3.score(predictors_test, target_test)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0.55625

# C. Conclusion

We are getting almost same results, if we expect accurate results, it's expected to run whole experiement multiple times and measure the variance in the results. We can select the final results.